In [ ]:
import os
import fitz
import re
import nltk
import pandas as pd
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords     
from nltk.util import ngrams                                    

In [ ]:
nltk.download("punkt")
nltk.download("stopwords")            

In [ ]:
input_folder = "Cancer2/"
output_folder = "25-3-2025_14_28/"
os.makedirs(output_folder, exist_ok=True)       

In [ ]:
unwanted_words = set([
    "article", "et", "al", "in", "to", "and", "for", "a", "y", "is", "of", "all", "the", "from", "are", "terms", "conditions", "publication", "citation",
    "open", "access", "license", "cc", "by", "creative", "commons", "attribution", "shown", "method", "state", "date", "plot", "trials", "per", "cent",
    "new", "present", "iii", "iv", "v", "etc", "proc", "natl", "acad", "sci", "usa", "vol", "pp", "using", "also", "used", "based", "may", "however", "one",
    "two", "three", "four", "five", "data", "set", "including", "due", "figure", "table", "fig", "found", "work", "among", "study", "analysis", "different",
    "several", "order", "low", "high", "higher", "lower", "within", "between", "without", "results", "approach", "across", "group", "suggest", "suggests",
    "indicate", "indicates", "according", "amongst", "even", "although", "further", "well", "known", "previously", "recent", "recently", "first", "second",
    "example", "examples", "others", "another", "obtained", "show", "shows", "would", "could", "can", "might", "many", "much", "certain", "some", "such",
    "particular", "often", "sometimes", "always", "never", "previous", "past", "future", "uncommon", "rare", "frequent", "other", "additional", "extra",
    "pros", "cons", "effective", "ineffective", "efficacy", "efficiency", "slow", "early", "late", "earlier", "latest", "delayed", "quick", "quicker",
    "quickest", "rapid", "rapidly", "slowly", "gradual", "sudden", "short", "long", "shorter", "shortest", "longer", "longest", "temporary", "permanent",
    "transient", "persistent", "mild", "moderate", "severe", "slight", "significant", "insignificant", "noticeable", "unnoticeable", "detectable", "wiley",
    "online", "library", "downloaded", "https", "see", "rules", "use", "oa", "articles", "governed", "applicable", "national", "center", "health",
    "statistics", "centers", "mortality", "public", "tapes", "american", "society", "atlanta", "tape", "reviews", "december", "volume", "nature",
    "publishing", "average", "annual", "percent", "change", "note", "trends", "analyzed", "joinpoint", "program", "total", "leading", "causes", "us",
    "standard", "end", "seer", "department", "individual", "b", "dvm", "phd", "year", "approximately", "fewer", "cases", "estimated", "alabama", "alaska",
    "arizona", "arkansas", "california", "colorado", "connecticut", "delaware", "dist", "columbia", "florida", "georgia", "hawaii", "idaho", "illinois",
    "indiana", "iowa", "kansas", "kentucky", "louisiana", "maine", "maryland", "massachusetts", "michigan", "minnesota", "mississippi", "missouri",
    "montana", "nebraska", "nevada", "hampshire", "jersey", "mexico", "york", "north", "carolina", "dakota", "ohio", "oklahoma", "oregon", "pennsylvania",
    "rhode", "island", "south", "tennessee", "texas", "utah", "vermont", "virginia", "washington", "west", "wisconsin", "wyoming", "ca", "clin", "j",
    "apc", "aapc", "ons", "mp", "cl", "ries", "lag", "eisner", "naaccr", "annu", "april", "author", "available", "c", "d", "e", "g", "h", "hruban", "i", "l",
    "maitra", "manuscript", "m", "n", "p", "page", "pathol", "pmc", "r", "rev", "s", "t", "u", "area", "as", "been", "by", "has", "have", "increased",
    "induced", "international", "mg", "meter", "not", "on", "reported", "research", "square", "sources", "studies", "substrate", "technol", "than", "that",
    "this", "topical", "under", "was", "with", "years", "younger", "journal", "notes", "tropic", "tr", "de", "carlos", "o", "publishers", "limited",
    "explore", "factor", "final", "edited", "there", "funders","x", "na", "number", "uses", "related", "text", "mining", "ai", "training", "similar", "guest", "website", "email", "info", "law", "lib", "personal",
    "only", "march", "february", "august", "july", "october", "june", "provided", "original", "properly", "cited", "http", "visit", "please", "repository",
    "more", "publications", "document", "identical", "content", "version", "postprint", "except", "adjusted", "normal", "dimension", "greatest", "cm",
    "you", "will", "be", "given", "note", "note:", "yes", "no", "line", "segment", "control", "strictinin", "or", "published", "academy", "advertisement",
    "ak", "az", "co", "dc", "begin", "blackadar", "charting", "combined", "copyright", "course", "day", "delivering", "director", "each", "exclude",
    "expected", "fl", "ga", "hi", "id", "ia", "guide", "had", "hallmarks", "hereby", "highest", "included", "inclusion", "interpreted", "january",
    "september", "november", "marked", "model", "must", "name", "offered", "occurrence", "office", "peak", "pointed", "printing", "pubmed", "rights",
    "rough", "should", "signs", "source", "suppl", "support", "system", "trend", "unexpected", "updated", "versus", "yes,", "no",
    "guest ","http"," march"," gut", "training","similar", "guest","http", "jnci" ,"no"," at ","university", "rounded ","nearest" , "excludes","basal", "rounded ","nearest"," exclude "," basal",
    "deaths each age do", "each age do sum", "age do sum ages", "do sum ages combined","aapck", "abdelmohsen", "adv", "ajcc", "appropriate", "artificial", "asr",
    "authors",  "committee", "competing", "credit", "csc", "curr", "current", "cumulative", "declare", "edited", "elmer", "error","europe", "exp", "explore", "factor", "final", "financial", "form",
    "funders", "inovação", "instituto", "interdiscip", "interests", "investigação", "intelligence", "jobin", "joint", "leone", "link", "manuscripts", "matulonis", "medium", "nat", "official", "opin", "our", "paper",
    "percentage", "porto", "powell", "press", "primers", "provide", "rank", "schwabe", "sites",
    "statement", "statements", "supp", "survival", "there", "were", "world"
    "o", "q", "í", "w", "f", "k", "md", "none","explor" , "ther" ,"int" , "pre"
])     

In [ ]:
def extract_text(pdf_file):
    doc = fitz.open(pdf_file)
    text = ""
    capture = False
    for page in doc:
        page_text = page.get_text()
        if "abstract" in page_text.lower():
            capture = True
        if "references" in page_text.lower():
            capture = False
        if capture:
            text += page_text + " "
    return text.lower() 

In [ ]:
global_ngram = {i: Counter() for i in range(1, 5)} 

In [ ]:
for file_name in os.listdir(input_folder):
    if file_name.endswith(".pdf"):
        pdf_path = os.path.join(input_folder, file_name)
        raw_text = extract_text(pdf_path)

        if not raw_text:
            continue

        tokens = word_tokenize(raw_text)
        filtered_tokens = [
            word for word in tokens
            if word.isalpha() and word not in unwanted_words and len(word) > 2
        ]

        for n in range(1, 5):
            global_ngram[n].update(Counter(ngrams(filtered_tokens, n))) 

In [ ]:
ngram_file_path = os.path.join(output_folder, "ngrams.txt")
with open(ngram_file_path, "w", encoding="utf-8") as f:
    for n in range(1, 5):
        f.write(f"\n{n}-grams:\n")
        for ngram, count in global_ngram[n].items():
            f.write(f"{' '.join(ngram)}: {count}\n")
        f.write("\n" + "=" * 50 + "\n") 

In [ ]:
import os
import re
import csv
from collections import defaultdict, Counter

file_path = "25-3-2025_14_28/ngrams.txt"
ngram_data = defaultdict(Counter)
pattern = re.compile(r'^(\d+)-grams:$')
current_n = 0

with open(file_path, "r", encoding="utf-8") as f:
    lines = f.readlines()

for line in lines:
    line = line.strip()
    if not line:
        continue
    match = pattern.match(line)
    if match:
        current_n = int(match.group(1))
        continue
    if current_n > 0 and ':' in line:
        ngram_str, count = line.rsplit(':', 1)
        ngram_tuple = tuple(ngram_str.strip().split())
        ngram_data[current_n][ngram_tuple] += int(count)

def get_subgrams(ngram, sub_len):
    return [tuple(ngram[i:i+sub_len]) for i in range(len(ngram) - sub_len + 1)]

for higher_n in range(4, 1, -1):  
    for higher_ngram, higher_count in ngram_data[higher_n].items():
        for sub_len in range(1, higher_n):
            for sub_ngram in get_subgrams(higher_ngram, sub_len):
                if sub_ngram in ngram_data[sub_len]:
                    ngram_data[sub_len][sub_ngram] -= higher_count

for n in list(ngram_data.keys()):
    ngram_data[n] = Counter({k: v for k, v in ngram_data[n].items() if v > 0})

output_dir = "25-3-2025_14_28"
os.makedirs(output_dir, exist_ok=True)

txt_path = os.path.join(output_dir, "clean_ngram.txt")
csv_path = os.path.join(output_dir, "clean_ngram.csv")

with open(txt_path, "w", encoding="utf-8") as txt_out, open(csv_path, "w", newline="", encoding="utf-8") as csv_out:
    writer = csv.writer(csv_out)
    writer.writerow(["N-gram", "Frequency"])
    for n in sorted(ngram_data):
        txt_out.write(f"\n{n}-grams:\n")
        for ngram, count in ngram_data[n].most_common():
            line = " ".join(ngram) + f": {count}"
            txt_out.write(line + "\n")
            writer.writerow([" ".join(ngram), count])

def save_top_clean_ngrams(filtered_ngrams, filename="top_1500_ngram.txt", top_n=1500):
    with open(filename, "w", encoding="utf-8") as f:
        f.write("Top 1500 Bigrams:\n")
        for ngram, count in filtered_ngrams[2].most_common(top_n):
            f.write(f"{' '.join(ngram)}: {count}\n")

        f.write("\nTop 1500 Trigrams:\n")
        for ngram, count in filtered_ngrams[3].most_common(top_n):
            f.write(f"{' '.join(ngram)}: {count}\n")

        f.write("\nTop 1500 Fourgrams:\n")
        for ngram, count in filtered_ngrams[4].most_common(top_n):
            f.write(f"{' '.join(ngram)}: {count}\n")

top_ngrams_1500_path = os.path.join(output_dir, "top_1500_ngram.txt")
save_top_clean_ngrams(ngram_data, filename=top_ngrams_1500_path, top_n=1500)   

In [ ]:
import os
import pandas as pd
from collections import Counter
from nltk import ngrams, word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import stopwords
from pdfminer.high_level import extract_text

def build_sparse_cooccurrence_csv(input_folder, top_ngrams_file, output_file):
    paper_files = [f for f in os.listdir(input_folder) if f.endswith(".pdf")]
    fourgrams = []

    with open(top_ngrams_file, "r", encoding="utf-8") as f:
        lines = f.readlines()
        capture = False
        for line in lines:
            if "Top 1500 Fourgrams:" in line:
                capture = True
                continue
            if capture and ":" in line:
                fourgram = line.split(":")[0].strip()
                if fourgram:
                    fourgrams.append(fourgram)
                if len(fourgrams) == 1500: 
                    break

    cooccurrence_data = []
    stop_words = set(stopwords.words("english"))

    for file_name in paper_files:
        pdf_path = os.path.join(input_folder, file_name)
        raw_text = extract_text(pdf_path)

        if not raw_text.strip():
            print(f"Skipped (no extractable text): {file_name}")
            continue

        tokens = [word for word in word_tokenize(raw_text.lower()) if word.isalpha() and word not in stop_words]
        paper_ngrams = Counter(ngrams(tokens, 4))

        for fourgram in fourgrams:
            fourgram_tuple = tuple(fourgram.split())
            count = paper_ngrams.get(fourgram_tuple, 0)
            if count > 0:
                cooccurrence_data.append((fourgram, file_name, count))

        print(f"Processed: {file_name}")

    df_sparse = pd.DataFrame(cooccurrence_data, columns=["Fourgram", "Paper", "Count"])
    df_sparse.to_csv(output_file, index=False)
    print(f"Co-occurrence matrix saved to: {output_file}") 

In [ ]:
input_folder = "Cancer2"
top_ngrams_file = "25-3-2025_14_28/top_1500_ngram.txt"
output_file = "25-3-2025_14_28/fourgram_cooccurrence.csv"

build_sparse_cooccurrence_csv(input_folder, top_ngrams_file, output_file)  

In [ ]:
import os
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD, LatentDirichletAllocation
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from pdfminer.high_level import extract_text

def load_top_1500_fourgrams(path):
    with open(path, "r", encoding="utf-8") as f:
        lines = f.readlines()
    fourgrams = []
    capture = False
    for line in lines:
        if "Top 1500 Fourgrams:" in line:
            capture = True
            continue
        if capture and ":" in line:
            fourgram = line.split(":")[0].strip()
            if fourgram:
                fourgrams.append(fourgram)
            if len(fourgrams) == 1500:
                break
    return fourgrams

def build_corpus(input_folder, top_fourgrams):
    files = [f for f in os.listdir(input_folder) if f.endswith(".pdf")]
    corpus = []
    for file in files:
        pdf_path = os.path.join(input_folder, file)
        text = extract_text(pdf_path)
        tokens = word_tokenize(text.lower())
        doc_ngrams = [" ".join(g) for g in ngrams(tokens, 4)]
        counts = Counter(doc_ngrams)
        filtered_doc = []
        for fg in top_fourgrams:
            filtered_doc.extend([fg] * counts.get(fg, 0))
        corpus.append(" ".join(filtered_doc))
    return corpus

def display_topics(model, feature_names, n_top_words=10):
    topics = []
    for topic_idx, topic in enumerate(model.components_):
        top_words = [feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        print(f"Topic {topic_idx+1}: {' | '.join(top_words)}")
        topics.append(top_words)
    return topics

def run_lsa_lda(input_folder, top_ngrams_file, num_topics=10):
    top_fourgrams = load_top_1500_fourgrams(top_ngrams_file)
    corpus = build_corpus(input_folder, top_fourgrams)

    tfidf_vectorizer = TfidfVectorizer(vocabulary=top_fourgrams)
    tfidf = tfidf_vectorizer.fit_transform(corpus)

    lsa = TruncatedSVD(n_components=num_topics, random_state=42)
    lsa.fit(tfidf)
    print("\n LSA Topics:")
    display_topics(lsa, tfidf_vectorizer.get_feature_names_out())

    count_vectorizer = CountVectorizer(vocabulary=top_fourgrams)
    count_data = count_vectorizer.fit_transform(corpus)

    lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
    lda.fit(count_data)
    print("\n LDA Topics:")
    display_topics(lda, count_vectorizer.get_feature_names_out())  

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from gensim import corpora, models
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
import os
from collections import Counter
from nltk.corpus import stopwords
from pdfminer.high_level import extract_text

def load_top_fourgrams(top_ngrams_file):
    fourgrams = []
    with open(top_ngrams_file, "r", encoding="utf-8") as f:
        lines = f.readlines()
        capture = False
        for line in lines:
            if "Top 1500 Fourgrams:" in line:
                capture = True
                continue
            if capture and ":" in line:
                fg = line.split(":")[0].strip()
                if fg:
                    fourgrams.append(" ".join(fg.split()))
                if len(fourgrams) == 1500:
                    break
    return fourgrams

def get_documents_from_pdfs(input_folder):
    stop_words = set(stopwords.words("english"))
    documents = []

    for file in sorted(os.listdir(input_folder)):
        if file.endswith(".pdf"):
            text = extract_text(os.path.join(input_folder, file))
            tokens = [t.lower() for t in word_tokenize(text) if t.isalpha() and t.lower() not in stop_words]
            documents.append(" ".join(tokens))
    return documents

def extract_top_fourgram_docs(documents, top_fourgrams):
    doc_texts = []
    for doc in documents:
        tokens = doc.split()
        doc_ngrams = [" ".join(ng) for ng in ngrams(tokens, 4)]
        filtered = [ng for ng in doc_ngrams if ng in top_fourgrams]
        doc_texts.append(" ".join(filtered))
    return doc_texts

def perform_lsa(documents, num_topics):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(documents)
    svd = TruncatedSVD(n_components=num_topics, random_state=42)
    lsa = svd.fit(tfidf_matrix)
    terms = vectorizer.get_feature_names_out()

    lsa_topics = []
    for i, comp in enumerate(lsa.components_):
        terms_in_topic = [terms[idx] for idx in comp.argsort()[:-11:-1]]
        lsa_topics.append([f"Topic {i+1}"] + terms_in_topic)
    return lsa_topics

def perform_lda(documents, num_topics):
    tokenized_docs = [doc.split() for doc in documents]
    dictionary = corpora.Dictionary(tokenized_docs)
    corpus = [dictionary.doc2bow(text) for text in tokenized_docs]
    lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10, random_state=42)

    lda_topics = []
    for i in range(num_topics):
        terms = [word for word, _ in lda_model.show_topic(i, topn=10)]
        lda_topics.append([f"Topic {i+1}"] + terms)
    return lda_topics

def save_topics_to_csv(lsa_topics, lda_topics, lsa_csv_file, lda_csv_file):
    lsa_df = pd.DataFrame(lsa_topics, columns=["Topic"] + [f"Term{i+1}" for i in range(10)])
    lda_df = pd.DataFrame(lda_topics, columns=["Topic"] + [f"Term{i+1}" for i in range(10)])

    lsa_df.to_csv(lsa_csv_file, index=False)
    lda_df.to_csv(lda_csv_file, index=False)

input_folder = "cancer2"
top_ngrams_file = "25-3-2025_14_28/top_1500_ngram.txt"
lsa_csv_file = "25-3-2025_14_28/lsa_topics.csv"
lda_csv_file = "25-3-2025_14_28/lda_topics.csv"
num_topics = 12

top_fourgrams = load_top_fourgrams(top_ngrams_file)
raw_documents = get_documents_from_pdfs(input_folder)
filtered_docs = extract_top_fourgram_docs(raw_documents, top_fourgrams)
lsa_topics = perform_lsa(filtered_docs, num_topics)
lda_topics = perform_lda(filtered_docs, num_topics)
save_topics_to_csv(lsa_topics, lda_topics, lsa_csv_file, lda_csv_file)

print(f"Saved LSA topics to {lsa_csv_file}")
print(f"Saved LDA topics to {lda_csv_file}") 

In [ ]:
import pandas as pd

lsa_df = pd.read_csv("25-3-2025_14_28/lsa_topics.csv")
lda_df = pd.read_csv("25-3-2025_14_28/lda_topics.csv")

topic_names = {
    "Topic 1": "Diagnosis",
    "Topic 2": "Cancer Cell Behavior",
    "Topic 3": "Expression Analysis",
    "Topic 4": "Tumor Characteristics",
    "Topic 5": "Tumor Biology",
    "Topic 6": "Cancer Therapy",
    "Topic 7": "Immune Response",
    "Topic 8": "Metastasis & Migration",
    "Topic 9": "Clinical Trials",
    "Topic 10": "Gene Expression",
    "Topic 11": "Side Effects",
    "Topic 12": "Survival & Prognosis"
}

lsa_df.insert(1, "Topic_Name", lsa_df["Topic"].map(topic_names))
lda_df.insert(1, "Topic_Name", lda_df["Topic"].map(topic_names))

lsa_df.to_csv("25-3-2025_14_28/lsa_topics_named.csv", index=False)
lda_df.to_csv("25-3-2025_14_28/lda_topics_named.csv", index=False)

print("✅ Named LSA topics saved to lsa_topics_named.csv")
print("✅ Named LDA topics saved to lda_topics_named.csv") 

In [ ]:
import pandas as pd

input_file = "25-3-2025_14_28/fourgram_cooccurrence.csv"
output_file = "25-3-2025_14_28/fourgram_jaccard_similarity_v3.csv"

df = pd.read_csv(input_file)
cooccurrence_matrix = df.pivot_table(index="Fourgram", columns="Paper", values="Count", fill_value=0)
binary_matrix = (cooccurrence_matrix > 0).astype(int)
jaccard_matrix = pd.DataFrame(index=binary_matrix.index, columns=binary_matrix.columns)

for paper in binary_matrix.columns:
    paper_set = set(binary_matrix[binary_matrix[paper] == 1].index)
    for fourgram in binary_matrix.index:
        fg_set = {fourgram}
        intersection = len(paper_set & fg_set)
        union = len(paper_set | fg_set)
        jaccard_score = intersection / union if union != 0 else 0
        jaccard_matrix.at[fourgram, paper] = jaccard_score

jaccard_matrix.to_csv(output_file)
print(f"Jaccard similarity matrix saved to: {output_file}")  

In [ ]:
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize
from nltk.corpus import stopwords
from pdfminer.high_level import extract_text

input_folder = "Cancer2"
ngram_file = "25-3-2025_14_28/top_1500_ngram.txt"
output_file = "25-3-2025_14_28/fourgram_tfidf.csv"

with open(ngram_file, "r", encoding="utf-8") as f:
    lines = f.readlines()
    fourgrams = []
    capture = False
    for line in lines:
        if "Top 1500 Fourgrams:" in line:
            capture = True
            continue
        if capture and ":" in line:
            fg = line.split(":")[0].strip()
            if fg:
                fourgrams.append(fg)
            if len(fourgrams) == 1500:
                break

paper_texts = {}
stop_words = set(stopwords.words("english"))

for file in os.listdir(input_folder):
    if file.endswith(".pdf"):
        path = os.path.join(input_folder, file)
        text = extract_text(path).lower()
        tokens = [w for w in word_tokenize(text) if w.isalpha() and w not in stop_words]
        paper_texts[file] = " ".join(tokens)

vectorizer = TfidfVectorizer(vocabulary=fourgrams, ngram_range=(4, 4), lowercase=True)
corpus = list(paper_texts.values())
tfidf_matrix = vectorizer.fit_transform(corpus)
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out(), index=paper_texts.keys())
df_tfidf = df_tfidf.T  
df_tfidf.to_csv(output_file)

print("✅ TF-IDF matrix saved to:", output_file)    

In [ ]:
import os
import pandas as pd
import spacy

base_path = "25-3-2025_14_28"
tfidf_path = os.path.join(base_path, "fourgram_tfidf.csv")
jaccard_path = os.path.join(base_path, "fourgram_jaccard_similarity_v3.csv")
cooccurrence_path = os.path.join(base_path, "fourgram_cooccurrence.csv")
lsa_topic_file = os.path.join(base_path, "lsa_topics.csv")
lda_topic_file = os.path.join(base_path, "lda_topics.csv")

nlp = spacy.load("en_core_web_sm")

tfidf_df = pd.read_csv(tfidf_path, index_col=0)
jaccard_df = pd.read_csv(jaccard_path, index_col=0)
cooccurrence_df = pd.read_csv(cooccurrence_path)

top_n = 300

top_tfidf = tfidf_df.mean(axis=1).sort_values(ascending=False).head(top_n).index.tolist()
jaccard_df["Avg_Jaccard"] = jaccard_df.mean(axis=1)
top_jaccard = jaccard_df[jaccard_df["Avg_Jaccard"] >= 0.1].index.tolist()
top_cooccur = (
    cooccurrence_df.groupby("Fourgram")["Count"]
    .sum()
    .sort_values(ascending=False)
    .head(top_n)
    .index
    .tolist()
)

combined_fourgrams = sorted(set(top_tfidf + top_jaccard + top_cooccur))

if os.path.exists(lsa_topic_file):
    lsa_df = pd.read_csv(lsa_topic_file)
    for col in lsa_df.columns:
        if "fourgram" in col.lower():
            combined_fourgrams += lsa_df[col].dropna().astype(str).unique().tolist()
            break

if os.path.exists(lda_topic_file):
    lda_df = pd.read_csv(lda_topic_file)
    for col in lda_df.columns:
        if "fourgram" in col.lower():
            combined_fourgrams += lda_df[col].dropna().astype(str).unique().tolist()
            break


combined_fourgrams = sorted(set(combined_fourgrams))

dependency_data = []
svo_list = []

for fg in combined_fourgrams:
    text = fg.replace("_", " ")
    doc = nlp(text)
    for token in doc:
        dependency_data.append({
            "Fourgram": fg,
            "Word": token.text,
            "Dependency": token.dep_,
            "Head": token.head.text,
            "Head_POS": token.head.pos_
        })
    subj, verb, obj = None, None, None
    for token in doc:
        if token.dep_ in {"nsubj", "nsubjpass"}:
            subj = token.text
        elif token.dep_ == "ROOT":
            verb = token.text
        elif token.dep_ in {"dobj", "pobj", "obj"}:
            obj = token.text
    if subj and verb and obj:
        svo_list.append({"Subject": subj, "Verb": verb, "Object": obj})

parsing_df = pd.DataFrame(dependency_data)
svo_df = pd.DataFrame(svo_list)

parsing_csv = os.path.join(base_path, "parsing.csv")   
svo_csv = os.path.join(base_path, "svooooooo.csv")

parsing_df.to_csv(parsing_csv, index=False)
svo_df.to_csv(svo_csv, index=False)

print("✅ Dependency parsing saved to:", parsing_csv)
print("✅ SVO triplets saved to:", svo_csv) 

In [ ]:
import pandas as pd
from neo4j import GraphDatabase

parsing_file = "25-3-2025_14_28/parsing.csv"   
svo_file = "25-3-2025_14_28/svooooooo.csv"

parsing_df = pd.read_csv(parsing_file)
svo_df = pd.read_csv(svo_file)

uri = "bolt://localhost:7687"
user = "neo4j"
password = "Sourabh@123"
driver = GraphDatabase.driver(uri, auth=(user, password))

def insert_dependency(tx, source, target, relation):
    tx.run("""
        MERGE (a:Word {text: $source})
        SET a.name = $source
        MERGE (b:Word {text: $target})
        SET b.name = $target
        MERGE (a)-[r:DEPENDS_ON]->(b)
        SET r.relation = $relation
    """, source=source, target=target, relation=relation)

def insert_svo(tx, subj, verb, obj):
    tx.run("""
        MERGE (s:Entity {text: $subj})
        SET s.name = $subj
        MERGE (o:Entity {text: $obj})
        SET o.name = $obj
        MERGE (s)-[r:RELATION]->(o)
        SET r.verb = $verb
    """, subj=subj, obj=obj, verb=verb)

with driver.session() as session:
    for _, row in parsing_df.iterrows():
        source = str(row.get("Word", "")).strip().lower()
        target = str(row.get("Head", "")).strip().lower()
        relation = str(row.get("Dependency", "")).strip().lower()
        if source and target and relation and "nan" not in [source, target, relation]:
            session.execute_write(insert_dependency, source, target, relation)

    for _, row in svo_df.iterrows():
        subj = str(row.get("Subject", "")).strip().lower()
        verb = str(row.get("Verb", "")).strip().lower()
        obj = str(row.get("Object", "")).strip().lower()
        if subj and verb and obj and "nan" not in [subj, verb, obj]:
            session.execute_write(insert_svo, subj, verb, obj)

driver.close() 

In [ ]:
import nest_asyncio
import threading
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel
from neo4j import GraphDatabase
from fastapi.responses import HTMLResponse

nest_asyncio.apply()

app = FastAPI()

uri = "bolt://localhost:7687"
user = "neo4j"
password = "Sourabh@123"
driver = GraphDatabase.driver(uri, auth=(user, password))

class SearchRequest(BaseModel):
    query: str

@app.get("/")
def root():
    return {"message": "Neo4j Dependency + SVO API is running"}

@app.get("/search-form", response_class=HTMLResponse)
def search_form():
    return """
    <!DOCTYPE html>
    <html>
    <head>
        <title>Neo4j Dependency + SVO Search</title>
        <style>
            body {
                font-family: 'Segoe UI', sans-serif;
                background-color: #eef6fc;
                padding: 40px;
                color: #003366;
            }
            h2, h3 {
                color: #005f99;
            }
            input, button {
                padding: 10px;
                font-size: 16px;
                margin-right: 10px;
                border: 1px solid #005f99;
                border-radius: 4px;
            }
            button {
                background-color: #005f99;
                color: white;
                cursor: pointer;
            }
            button:hover {
                background-color: #004c80;
            }
            pre {
                background-color: #ffffff;
                padding: 15px;
                border: 1px solid #cce0ff;
                margin-top: 10px;
                white-space: pre-wrap;
                border-radius: 5px;
            }
            .section {
                margin-top: 30px;
            }
            #loading {
                display: none;
                font-style: italic;
                color: #555;
            }
        </style>
    </head>
    <body>
        <h2>Neo4j Dependency and SVO Search</h2>
        <form id="searchForm">
            <input type="text" id="query" placeholder="Enter a keyword..." required>
            <button type="submit">Search</button>
            <span id="loading">Loading...</span>
        </form>

        <div class="section">
            <h3>Dependency Relations</h3>
            <pre id="depsFriendly"></pre>
            <pre id="depsRaw"></pre>
        </div>

        <div class="section">
            <h3>SVO Relations</h3>
            <pre id="svoFriendly"></pre>
            <pre id="svoRaw"></pre>
        </div>

        <script>
            const relationMap = {
                "nsubj": "is the subject of",
                "obj": "is the object of",
                "compound": "is a part of",
                "amod": "is described by",
                "advmod": "modifies (adverbially)",
                "prep": "is related via preposition to",
                "conj": "is connected with",
                "poss": "is possessed by",
                "det": "has a determiner",
                "nmod": "has a modifier",
                "acl": "has a clause modifying it",
                "case": "is linked by a case marker"
            };

            const form = document.getElementById('searchForm');
            const loading = document.getElementById('loading');

            form.onsubmit = async (e) => {
                e.preventDefault();
                const query = document.getElementById('query').value;
                loading.style.display = "inline";

                const depResponse = await fetch('/dependencies/search', {
                    method: 'POST',
                    headers: {'Content-Type': 'application/json'},
                    body: JSON.stringify({ query })
                });
                const svoResponse = await fetch('/svo/search', {
                    method: 'POST',
                    headers: {'Content-Type': 'application/json'},
                    body: JSON.stringify({ query })
                });

                const deps = await depResponse.json();
                const svos = await svoResponse.json();

                document.getElementById('depsRaw').textContent = JSON.stringify(deps, null, 2);
                document.getElementById('svoRaw').textContent = JSON.stringify(svos, null, 2);

                document.getElementById('depsFriendly').textContent = deps.map(d => {
                    const readable = relationMap[d.relation] || `is related to (via "${d.relation}")`;
                    return `${capitalize(d.source)} ${readable} ${capitalize(d.target)}`;
                }).join("\\n");

                document.getElementById('svoFriendly').textContent = svos.map(d => {
                    return `${capitalize(d.subject)} ${d.verb} ${capitalize(d.object)}`;
                }).join("\\n");

                loading.style.display = "none";
            };

            function capitalize(word) {
                return word.charAt(0).toUpperCase() + word.slice(1);
            }
        </script>
    </body>
    </html>
    """

@app.post("/dependencies/search")
def search_dependencies(req: SearchRequest):
    with driver.session() as session:
        result = session.run("""
            MATCH (a:Word)-[r:DEPENDS_ON]->(b:Word)
            WHERE toLower(a.text) CONTAINS toLower($query)
               OR toLower(b.text) CONTAINS toLower($query)
               OR toLower(r.relation) CONTAINS toLower($query)
            RETURN a.text AS source, r.relation AS relation, b.text AS target
        """, {"query": req.query})
        return [record.data() for record in result]

@app.post("/svo/search")
def search_svo(req: SearchRequest):
    with driver.session() as session:
        result = session.run("""
            MATCH (s:Entity)-[r:RELATION]->(o:Entity)
            WHERE toLower(s.text) CONTAINS toLower($query)
               OR toLower(o.text) CONTAINS toLower($query)
               OR toLower(r.verb) CONTAINS toLower($query)
            RETURN s.text AS subject, r.verb AS verb, o.text AS object
        """, {"query": req.query})
        return [record.data() for record in result]

def run_app():
    uvicorn.run(app, host="127.0.0.1", port=8000)

threading.Thread(target=run_app).start()   

In [ ]:
import os
import pandas as pd
import spacy

base_path = "25-3-2025_14_28"
tfidf_path = os.path.join(base_path, "fourgram_tfidf.csv")
jaccard_path = os.path.join(base_path, "fourgram_jaccard_similarity_v3.csv")
cooccurrence_path = os.path.join(base_path, "fourgram_cooccurrence.csv")
lsa_topic_file = os.path.join(base_path, "lsa_topics.csv")
lda_topic_file = os.path.join(base_path, "lda_topics.csv")

nlp = spacy.load("en_core_sci_sm")  

tfidf_df = pd.read_csv(tfidf_path, index_col=0)
jaccard_df = pd.read_csv(jaccard_path, index_col=0)
cooccurrence_df = pd.read_csv(cooccurrence_path)

top_n = 750

top_tfidf = tfidf_df.mean(axis=1).sort_values(ascending=False).head(top_n).index.tolist()
jaccard_df["Avg_Jaccard"] = jaccard_df.mean(axis=1)
top_jaccard = jaccard_df[jaccard_df["Avg_Jaccard"] >= 0.1].index.tolist()
top_cooccur = (
    cooccurrence_df.groupby("Fourgram")["Count"]
    .sum()
    .sort_values(ascending=False)
    .head(top_n)
    .index
    .tolist()
)

combined_fourgrams = sorted(set(top_tfidf + top_jaccard + top_cooccur))

if os.path.exists(lsa_topic_file):
    lsa_df = pd.read_csv(lsa_topic_file)
    for col in lsa_df.columns:
        if "fourgram" in col.lower():
            combined_fourgrams += lsa_df[col].dropna().astype(str).unique().tolist()
            break

if os.path.exists(lda_topic_file):
    lda_df = pd.read_csv(lda_topic_file)
    for col in lda_df.columns:
        if "fourgram" in col.lower():
            combined_fourgrams += lda_df[col].dropna().astype(str).unique().tolist()
            break

combined_fourgrams = sorted(set(combined_fourgrams))

dependency_data = []
valid_relations = []  

def is_complete_word(word):
    return len(word) > 1 and word.isalpha()

def is_valid_relation(subj, verb, obj):
    return len(set([subj, verb, obj])) == 3  

for fg in combined_fourgrams:
    text = fg.replace("_", " ")
    doc = nlp(text)

    for token in doc:
        dependency_data.append({
            "Fourgram": fg,
            "Word": token.text,
            "Dependency": token.dep_,
            "Head": token.head.text,
            "Head_POS": token.head.pos_
        })

    subj, verb, obj = None, None, None
    
    for token in doc:
        if token.dep_ in {"nsubj", "nsubjpass"} and is_complete_word(token.text):
            subj = token.text
        elif token.dep_ == "ROOT" and is_complete_word(token.text):
            verb = token.text
        elif token.dep_ in {"dobj", "pobj", "obj"} and is_complete_word(token.text):
            obj = token.text

    if subj and verb and obj:
        if is_valid_relation(subj, verb, obj) and "is part of" not in [subj, verb, obj] and "of" not in [subj, verb, obj]:
            valid_relations.append({"Subject": subj, "Verb": verb, "Object": obj})

parsing_df = pd.DataFrame(dependency_data)
relations_df = pd.DataFrame(valid_relations)

parsing_csv = os.path.join(base_path, "parsing.csv")
relations_csv = os.path.join(base_path, "relations_sci.csv")

parsing_df.to_csv(parsing_csv, index=False)
relations_df.to_csv(relations_csv, index=False)

print("✅ Dependency parsing saved to:", parsing_csv)
print("✅ Valid scientific relations saved to:", relations_csv) 

In [ ]:
import pandas as pd
from neo4j import GraphDatabase

parsing_file = "25-3-2025_14_28/parsing.csv"   
relations_file = "25-3-2025_14_28/relations_sci.csv"  

parsing_df = pd.read_csv(parsing_file)
relations_df = pd.read_csv(relations_file)

uri = "bolt://localhost:7687"
user = "neo4j"
password = "sourabh@123"
driver = GraphDatabase.driver(uri, auth=(user, password))

def insert_dependency(tx, source, target, relation):
    tx.run("""
        MERGE (a:Word {text: $source})
        SET a.name = $source
        MERGE (b:Word {text: $target})
        SET b.name = $target
        MERGE (a)-[r:DEPENDS_ON]->(b)
        SET r.relation = $relation
    """, source=source, target=target, relation=relation)

def insert_svo(tx, subj, verb, obj):
    tx.run("""
        MERGE (s:Entity {text: $subj})
        SET s.name = $subj
        MERGE (o:Entity {text: $obj})
        SET o.name = $obj
        MERGE (s)-[r:RELATION]->(o)
        SET r.verb = $verb
    """, subj=subj, obj=obj, verb=verb)

with driver.session() as session:
    for _, row in parsing_df.iterrows():
        source = str(row.get("Word", "")).strip().lower()
        target = str(row.get("Head", "")).strip().lower()
        relation = str(row.get("Dependency", "")).strip().lower()
        
        if source and target and relation and "nan" not in [source, target, relation]:
            session.execute_write(insert_dependency, source, target, relation)

    for _, row in relations_df.iterrows():
        subj = str(row.get("Subject", "")).strip().lower()
        verb = str(row.get("Verb", "")).strip().lower()
        obj = str(row.get("Object", "")).strip().lower()
        
        if subj and verb and obj and "nan" not in [subj, verb, obj]:
            session.execute_write(insert_svo, subj, verb, obj)

driver.close() 

In [ ]:
import nest_asyncio
import threading
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel
from neo4j import GraphDatabase
from fastapi.responses import HTMLResponse

nest_asyncio.apply()

app = FastAPI()

uri = "bolt://localhost:7687"
user = "neo4j"
password = "sourabh@123"
driver = GraphDatabase.driver(uri, auth=(user, password))

class SearchRequest(BaseModel):
    query: str

@app.get("/")
def root():
    return {"message": "Neo4j Dependency + SVO API is running"}

@app.get("/search-form", response_class=HTMLResponse)
def search_form():
    return """
    <!DOCTYPE html>
    <html>
    <head>
        <title>Neo4j Dependency + SVO Search</title>
        <style>
            body {
                font-family: 'Segoe UI', sans-serif;
                background-color: #eef6fc;
                padding: 40px;
                color: #003366;
            }
            h2, h3 {
                color: #005f99;
            }
            input, button {
                padding: 10px;
                font-size: 16px;
                margin-right: 10px;
                border: 1px solid #005f99;
                border-radius: 4px;
            }
            button {
                background-color: #005f99;
                color: white;
                cursor: pointer;
            }
            button:hover {
                background-color: #004c80;
            }
            pre {
                background-color: #ffffff;
                padding: 15px;
                border: 1px solid #cce0ff;
                margin-top: 10px;
                white-space: pre-wrap;
                border-radius: 5px;
            }
            .section {
                margin-top: 30px;
            }
            #loading {
                display: none;
                font-style: italic;
                color: #555;
            }
        </style>
    </head>
    <body>
        <h2>Neo4j Dependency and SVO Search</h2>
        <form id="searchForm">
            <input type="text" id="query" placeholder="Enter a keyword..." required>
            <button type="submit">Search</button>
            <span id="loading">Loading...</span>
        </form>

        <div class="section">
            <h3>Dependency Relations</h3>
            <pre id="depsFriendly"></pre>
            <pre id="depsRaw"></pre>
        </div>

        <div class="section">
            <h3>SVO Relations</h3>
            <pre id="svoFriendly"></pre>
            <pre id="svoRaw"></pre>
        </div>

        <script>
            const relationMap = {
                "nsubj": "is the subject of",
                "obj": "is the object of",
                "compound": "is a part of",
                "amod": "is described by",
                "advmod": "modifies (adverbially)",
                "prep": "is related via preposition to",
                "conj": "is connected with",
                "poss": "is possessed by",
                "det": "has a determiner",
                "nmod": "has a modifier",
                "acl": "has a clause modifying it",
                "case": "is linked by a case marker"
            };

            const form = document.getElementById('searchForm');
            const loading = document.getElementById('loading');

            form.onsubmit = async (e) => {
                e.preventDefault();
                const query = document.getElementById('query').value;
                loading.style.display = "inline";

                const depResponse = await fetch('/dependencies/search', {
                    method: 'POST',
                    headers: {'Content-Type': 'application/json'},
                    body: JSON.stringify({ query })
                });
                const svoResponse = await fetch('/svo/search', {
                    method: 'POST',
                    headers: {'Content-Type': 'application/json'},
                    body: JSON.stringify({ query })
                });

                const deps = await depResponse.json();
                const svos = await svoResponse.json();

                document.getElementById('depsRaw').textContent = JSON.stringify(deps, null, 2);
                document.getElementById('svoRaw').textContent = JSON.stringify(svos, null, 2);

                document.getElementById('depsFriendly').textContent = deps.map(d => {
                    const readable = relationMap[d.relation] || `is related to (via "${d.relation}")`;
                    return `${capitalize(d.source)} ${readable} ${capitalize(d.target)}`;
                }).join("\\n");

                document.getElementById('svoFriendly').textContent = svos.map(d => {
                    return `${capitalize(d.subject)} ${d.verb} ${capitalize(d.object)}`;
                }).join("\\n");

                loading.style.display = "none";
            };

            function capitalize(word) {
                return word.charAt(0).toUpperCase() + word.slice(1);
            }
        </script>
    </body>
    </html>
    """

@app.post("/dependencies/search")
def search_dependencies(req: SearchRequest):
    with driver.session() as session:
        result = session.run("""
            MATCH (a:Word)-[r:DEPENDS_ON]->(b:Word)
            WHERE toLower(a.text) CONTAINS toLower($query)
               OR toLower(b.text) CONTAINS toLower($query)
               OR toLower(r.relation) CONTAINS toLower($query)
            RETURN a.text AS source, r.relation AS relation, b.text AS target
        """, {"query": req.query})
        return [record.data() for record in result]

@app.post("/svo/search")
def search_svo(req: SearchRequest):
    with driver.session() as session:
        result = session.run("""
            MATCH (s:Entity)-[r:RELATION]->(o:Entity)
            WHERE toLower(s.text) CONTAINS toLower($query)
               OR toLower(o.text) CONTAINS toLower($query)
               OR toLower(r.verb) CONTAINS toLower($query)
            RETURN s.text AS subject, r.verb AS verb, o.text AS object
        """, {"query": req.query})
        return [record.data() for record in result]

def run_app():
    uvicorn.run(app, host="127.0.0.1", port=8000)

threading.Thread(target=run_app).start() 